In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from data_providers import *
from utils.utils import AverageMeter
from thop import profile

In [5]:
import pandas as pd
pd.read_csv('harblocks.csv')

,dataset,batch,window_size,block,kernel,input_channels,out_channels,num_layers,hardware,device,latency
0,uci,1,128,LSTMBlock,0,32,32,1,pc,cpu,0.001637
1,uci,1,128,LSTMBlock,0,32,32,1,pc,cpu,0.001720
2,uci,1,128,LSTMBlock,0,32,32,1,pc,cpu,0.001556
3,uci,1,128,LSTMBlock,0,32,32,1,pc,cpu,0.001666
4,uci,1,128,LSTMBlock,0,32,32,1,pc,cpu,0.001603
...,...,...,...,...,...,...,...,...,...,...,...
95,uci,1,128,BiLSTMBlock,0,32,128,1,pc,gpu,0.692719
96,uci,1,128,BiLSTMBlock,0,32,128,1,pc,gpu,0.692243
97,uci,1,128,BiLSTMBlock,0,32,128,1,pc,gpu,0.701003
98,uci,1,128,BiLSTMBlock,0,32,128,1,pc,gpu,0.706901


# All Blocks

In [2]:
block_list = ['LSTMBlock', 'BiLSTMBlock', 'GTSResConvBlock']
lstm_block_list = ['LSTMBlock', 'BiLSTMBlock']

* cpu

In [3]:
for block in block_list:
    for _ in range(10):
        ! python measure_blocks/measure_harblock_latency.py --hardware pc --device cpu --dataset uci --block_name $block --num-runs 100 --config_file harblocks.csv
        
for block in lstm_block_list:
    for _ in range(10):
        ! python measure_blocks/measure_harblock_latency.py --hardware pc --device cpu --dataset uci --block_name $block --out_channels 128 --num-runs 100 --config_file harblocks.csv

LSTMBlock: 0.001637
Experiment results saved to harblocks.csv
LSTMBlock: 0.001720
Experiment results saved to harblocks.csv
LSTMBlock: 0.001556
Experiment results saved to harblocks.csv
LSTMBlock: 0.001666
Experiment results saved to harblocks.csv
LSTMBlock: 0.001603
Experiment results saved to harblocks.csv
LSTMBlock: 0.001648
Experiment results saved to harblocks.csv
LSTMBlock: 0.001690
Experiment results saved to harblocks.csv
LSTMBlock: 0.001667
Experiment results saved to harblocks.csv
LSTMBlock: 0.001676
Experiment results saved to harblocks.csv
LSTMBlock: 0.001626
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001728
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001786
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001699
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001644
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001713
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.001644
Experiment results saved to harblocks.

* gpu

In [4]:
for block in block_list:
    for _ in range(10):
        ! python measure_blocks/measure_harblock_latency.py --hardware pc --device gpu --dataset uci --block_name $block --num-runs 100 --config_file harblocks.csv
        
for block in lstm_block_list:
    for _ in range(10):
        ! python measure_blocks/measure_harblock_latency.py --hardware pc --device gpu --dataset uci --block_name $block --out_channels 128 --num-runs 100 --config_file harblocks.csv

LSTMBlock: 0.085980
Experiment results saved to harblocks.csv
LSTMBlock: 0.085437
Experiment results saved to harblocks.csv
LSTMBlock: 0.085433
Experiment results saved to harblocks.csv
LSTMBlock: 0.085749
Experiment results saved to harblocks.csv
LSTMBlock: 0.085524
Experiment results saved to harblocks.csv
LSTMBlock: 0.086058
Experiment results saved to harblocks.csv
LSTMBlock: 0.086566
Experiment results saved to harblocks.csv
LSTMBlock: 0.085980
Experiment results saved to harblocks.csv
LSTMBlock: 0.086241
Experiment results saved to harblocks.csv
LSTMBlock: 0.085933
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.162778
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.163076
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.163810
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.163649
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.163552
Experiment results saved to harblocks.csv
BiLSTMBlock: 0.163891
Experiment results saved to harblocks.

* ETC

In [147]:
def get_bin_dist(input_tensor, num_bins=10):
    min_value, _ = input_tensor.min(dim=2)
    max_value, _ = input_tensor.max(dim=2)
    
    fractions = torch.empty(input_tensor.shape[0], input_tensor.shape[1], num_bins).to(input_tensor.device)   # batch, num_bins
    for i in range(min_value.shape[-1]):
        histogram = torch.histc(input_tensor[0][i], bins=num_bins, min=min_value[0][i], max=max_value[0][i])
        fractions[0][i] = histogram / input_tensor.shape[-1]

    return fractions.reshape(-1, input_tensor.shape[1]*num_bins)

def get_stats_feats(input_tensor, num_bins=10):
    avg = torch.mean(input_tensor, dim=2)
    std = torch.std(input_tensor, dim=2)
    avg_abs_diff = torch.abs(input_tensor - axis_avg.unsqueeze(2)).mean(dim=2)
    avg_resultant = input_tensor.pow(2).sum(dim=1).sqrt().mean(dim=1).unsqueeze(1)
    bin_dist = get_bin_dist(input_tensor, num_bins=10)
    feats = torch.cat([avg, std, avg_abs_diff, avg_resultant, bin_dist], dim=1)
    return feats

In [145]:
avg = torch.mean(input_tensor, dim=2)
std = torch.std(input_tensor, dim=2)
avg_abs_diff = torch.abs(input_tensor - axis_avg.unsqueeze(2)).mean(dim=2)
avg_resultant = input_tensor.pow(2).sum(dim=1).sqrt().mean(dim=1).unsqueeze(1)
bin_dist = get_bin_dist(input_tensor, num_bins=10)

In [149]:
get_stats_feats(input_tensor).shape

torch.Size([1, 79])

In [146]:
axis_avg.shape, axis_std.shape, axis_avg_abs_diff.shape, avg_resultant.shape, bin_dist.shape

(torch.Size([1, 6]),
 torch.Size([1, 6]),
 torch.Size([1, 6]),
 torch.Size([1, 1]),
 torch.Size([1, 60]))

In [36]:
class BasicConv1d(nn.Module):
    def __init__(self, i_nc, o_nc, **kwargs):
        super(BasicConv1d, self).__init__()
        self.conv = nn.Conv1d(i_nc, o_nc, **kwargs)
    def forward(self, x):
        x = self.conv(x)
        return F.relu_(x)

class RTCNN(nn.Module):
    def __init__(self, window_size, init_channels, num_classes, acc_num):
        super(RTCNN, self).__init__()
        self.accs = acc_num
        
        self.conv = BasicConv1d(init_channels, 196, kernel_size=16, stride=1, padding=8)
        self.maxpool = nn.MaxPool1d(kernel_size=4, stride=4, padding=0)

        self.fc = nn.Sequential(
            nn.Linear(flat_size + 40*acc_num, 1024),
            nn.ReLU(),
            nn.Dropout(0.05),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        stats = get_stats_feats(x)
        x = self.conv(x)
        x = self.maxpool(x)

        x = x.view(x.size(0), -1)
        x = torch.cat([x, stats], dim=1)
        logits = self.fc(x)

        return logits

torch.return_types.min(
values=tensor([[-3.0980, -2.5481, -3.6223, -2.4390, -3.1402, -3.5049]]),
indices=tensor([[ 51, 122,  61,   5, 111,  74]]))